<a href="https://colab.research.google.com/github/judeha/addnoise/blob/main/Calculate_Subst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Drive setup
from google.colab import drive, files
drive.mount('/content/drive/')

path="drive/My Drive/Dell Lab/"

Mounted at /content/drive/


In [17]:
# Import dependencies
import numpy as np
import pandas as pd
import math
import json
import os

In [ ]:
# Import error data from drive 
fp = path + 'error_counts.npy'

counts = np.load(fp)

a_subs = counts[ord('a')]

for i, x in enumerate(a_subs):
    if x != 0:
        print('{}: {}'.format(chr(i), x))

In [ ]:
# Import homoglyph list | not necessary anymore
f = open(path + "homoglyph_list.json")

homoglyph = pd.read_json(f)
homoglyph = homoglyph.sort_values(by=["a","b"])
print(homoglyph)

In [ ]:
# Import and reshape homoglyph list
lst = pd.read_csv(path + 'homoglyph.csv')
lst = lst.drop('Unnamed: 0', axis=1)

df = pd.DataFrame(lst)
homoglyph = pd.pivot_table(df, index='a', columns='b', values='sim_score', fill_value=0)

print(homoglyph)

In [ ]:
# Calculate probability of substitution
import statistics as stats
from math import isnan

err = 0.05

# find sums of similarities 
sums = []

# NOTE: not sure why iterrows doesn't work here
for i in range(len(homoglyph.index)):
  row = homoglyph.iloc[i]
  lst = row.values.tolist()
  lst = [x for x in lst if isnan(x)==False]
  sums.append(sum(lst))

print(sums)
# standardize using max sum
mx = max(sums)

def helper(n):
  return err * (n/mx)

subst_prob = list(map(helper,sums))
print(subst_prob)

In [ ]:
# Convert back to dataframe with labels
labels = homoglyph.columns.values.tolist()
arr = [labels, subst_prob]

ref = pd.DataFrame(arr, columns=labels)
ref.fillna(0)
print(ref)

In [ ]:
# Normalize similarity scores to get CDFs
def find_max(r):
  return r.max(axis=0)

def normalize(n, factor):
  return n / factor

homoglyph = homoglyph.apply(lambda row : normalize(row, find_max(row)), axis=1)
print(homoglyph)

In [ ]:
# %reset_selective -f "homoglyph"

In [24]:
from re import S
from prompt_toolkit.layout.dummy import D
# Sort similarity scores by CDFs for each char
# Construct dict of dict of k,v pairs

subst_cdf = {}
for r in range(len(homoglyph.index)):
  Dict = {}
  for c in range(len(homoglyph.columns)):
    Dict.update({homoglyph.iloc[r][c]: homoglyph.columns.values[c]})
  s=dict(sorted(Dict.items()))
  subst_cdf.update({homoglyph.index[r]: s})

In [30]:
subst_cdf = json.dumps(subst_cdf)
subst_prob = ref.to_csv()

with open('/content/drive/My Drive/Dell Lab/subst_cdf.json', 'w') as f:
  f.write(subst_cdf)

with open('/content/drive/My Drive/Dell Lab/subst_prob.csv', 'w') as f:
  f.write(subst_prob)